# Máltækni - Náttúruvernd

# Data import & manipulation

In [108]:
from numpy.random import default_rng
import re
import tqdm
import pandas as pd
import pickle
import numpy as np

In [105]:
#@Vésteinn
def read_news_tsv(file_name):
    data = []
    with open(file_name, encoding="utf-8") as fh:
        for line in tqdm.tqdm(fh.readlines()):
            title, author, url, timestamp, _, _, text, text_tokenized = line.split("\t")
            article = {
                "title": title,
                "author": author,
                "timestamp": timestamp,
                "text": text,
                "text_tokenized": text_tokenized
            }
            data.append(article)
        return data

In [8]:
articles = read_news_tsv("icelandic_news_sample_22480.tsv")
print(f"Article count {len(articles)}")

100%|████████████████████████████████████████████████████████████████████████| 22248/22248 [00:00<00:00, 195903.43it/s]

Article count 22248


In [12]:
articles_df = pd.DataFrame(articles)

In [22]:
articles_df.head()

,title,author,timestamp,text,text_tokenized
0,ARON CAN ER SILKISLAKUR OG SVALUR - Albumm,Ritstjórn visir.is,2017-03-13 12:30:11.58353,ARON CAN ER SILKISLAKUR OG SVALUR—Albumm ☰ FRÉ...,ARON CAN ER SILKISLAKUR OG SVALUR — Albumm ☰ F...
1,Sveindís kemur mjög vel inn í okkar hóp,Sindri Sverrisson,2020-09-16 13:00:00,"Jón Þór Hauksson, landsliðsþjálfari í fótbolta...","Jón Þór Hauksson , landsliðsþjálfari í fótbolt..."
2,Keyrði á kyrrstæðan bíl í miðbænum: „Gríðarleg...,Ritstjórn DV,2019-06-17 08:15:05,Karlmaður var fluttur á slysadeild í nótt efti...,Karlmaður var fluttur á slysadeild í nótt efti...
3,Markaðssetja þurfi vannýtt svæði á landsbyggðinni,Margrét Helga Erlingsdóttir,2019-09-20 12:59:00,Stjórnvöld verða að móta sér stefnu til framtí...,Stjórnvöld verða að móta sér stefnu til framtí...
4,Biður um þolinmæði í skertri velferðarþjónustu,Þórdís Arnljótsdóttir,2020-03-28 19:47:25,Stór hluti þeirra sem fær velferðarþjónustu í ...,Stór hluti þeirra sem fær velferðarþjónustu í ...


In [13]:
articles_Big = read_news_tsv("articles_until_01-11-2020_detok.tsv")
print(f"Article count {len(articles_Big)}")

100%|██████████████████████████████████████████████████████████████████████| 640136/640136 [00:05<00:00, 111343.55it/s]


Article count 640136


In [14]:
news_Big = pd.DataFrame(articles_Big)

In [15]:
news_Big['text'].isnull().sum()

0

In [16]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(news_Big, test_size = 0.15, random_state=11)

In [17]:
print(f'Training data examles: {data_train.shape[0]}')
print(f'Test data examles: {data_test.shape[0]}')

Training data examles: 544115
Test data examles: 96021


In [18]:
data_test, data_test_labeled = train_test_split(data_test, test_size = 0.005, random_state=11)

In [19]:
print(f'Test data examles: {data_test.shape[0]}')
print(f'Labeled test data examles: {data_test_labeled.shape[0]}')

Test data examles: 95540
Labeled test data examles: 481


In [20]:
# Subset af training til að koma virkni í gang án þjálfunartíma
_, data_playground = train_test_split(data_train, test_size = 0.10, random_state=11)
print(f'Playground data examles: {data_playground.shape[0]}')

Playground data examles: 54412


# Skriftupælingar

In [23]:
def rand_frett(safn, count):
    listi = []
    from numpy.random import default_rng
    import re
    rng = default_rng()
    n = safn.shape[0]
    for i in tqdm.trange(count):
        x = rng.integers(n)
        if pd.isnull(safn.iloc[x]["text"]):
            continue
        frett = safn.iloc[x]["text"] # ath iloc
        frettnr = safn.iloc[x].name
        lykilord = ["náttúruvernd", "loftslag", "umhverfi"]
        fjoldi = 0
        for wrd in lykilord:
            fjoldi += len(re.findall(wrd, frett))
        if fjoldi > 0:
            listi.append(frettnr) # Ath. algildur index
            #print(frett)
    return listi

    

In [128]:
def allar_frettir(safn, threshold=0):
    listi = []

    n = safn.shape[0]
    for i in tqdm.trange(n):
        frett = safn.iloc[i]["text"]
        lykilord = ["náttúruvernd", "loftslag", "umhverfi"]
        fjoldi = 0
        for wrd in lykilord:
            fjoldi += len(re.findall(wrd, frett))
        if fjoldi > threshold:
            listi.append(i) # ath. afstæð númer m.v. safn
            #print(frett)
    return listi

    

In [25]:
sample_count = 100000
big_list = rand_frett(news_Big, sample_count)
print(len(big_list)/sample_count)

100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [00:07<00:00, 13338.91it/s]

0.06751


In [26]:
sigtaðar_frettir = allar_frettir(data_test_labeled)

100%|█████████████████████████████████████████████████████████████████████████████| 481/481 [00:00<00:00, 10546.11it/s]


In [ ]:
for fr in sigtaðar_frettir[:10]:
    print(data_test_labeled.iloc[fr])

In [28]:
len(sigtaðar_frettir)

38

In [ ]:
for fr in big_list[:10]:
    print(news_Big.loc[fr])

In [ ]:
ja = []
vafi = []
annad = []
for i in sigtaðar_frettir[:3]:
    frett = data_test_labeled.iloc[i]
    print(frett["text"])
    print(frett["title"])
    einkunn = input("Um náttúruvernd?\n")
    if einkunn == str(1):
        ja.append(frett.name)
    elif einkunn == str(2):
        vafi.append(frett.name)
    else:
        annad.append(frett.name)

In [133]:
def label(safn, sigti, count):
    ja = []
    vafi = []
    annad = []
    for i in sigti[:count]:
        frett = safn.iloc[i]
        print(frett["text"])
        print(frett["title"])
        einkunn = input("Um náttúruvernd?\n")
        print()
        if einkunn == str(1):
            ja.append(frett.name)
        elif einkunn == str(2):
            vafi.append(frett.name)
        else:
            annad.append(frett.name)
    return ja, vafi, annad

In [83]:
jakvætt = set([165133,322727,343386,233961,500609,327615])
neikvætt = set(data_test_labeled.index).difference(jakvætt)

print(len(neikvætt))
print(len(jakvætt))
print(data_test_labeled.shape[0])

475
6
481


In [111]:
data_test_labeled["label"] = np.zeros((len(data_test_labeled), 1), dtype = np.int8)

In [122]:
for i in jakvætt:
    data_test_labeled.loc[i,"label"] = 1

In [135]:
playground_sigti = allar_frettir(data_playground, 5)

100%|█████████████████████████████████████████████████████████████████████████| 54412/54412 [00:01<00:00, 30996.20it/s]


In [136]:
len(playground_sigti)

229

In [ ]:
ja_p, kannski_p, nei_p = label(data_playground, playground_sigti, 20)

# GENSIM LDA pælingar

In [140]:
import gensim

In [141]:
import string
stop_words = set()
extra_stop_words = ["var", "ver", "ekki", "sé"]
with open("is_stop_words.txt") as stop_word_file:
    for line in stop_word_file.readlines():
        stop_words.add(line.strip())
stop_words.update(set(string.punctuation))
stop_words.update(set(extra_stop_words))

In [142]:
import tokenizer
from islenska import Bin

b = Bin()

def naive_bin_lemma(token):
    candidates = b.lookup_lemmas_and_cats(token)
    if not candidates:
        return None
    if len(candidates) > 1:
        return None
    cand = candidates.pop()
    if cand[1] not in ["kk", "kvk", "hk"]:
        return None
    return cand[0]


def tokenize_and_lemmatize(sentence):
    if pd.isnull(sentence): # Líklegast óþarft núna
            return []
    tokenized_text = list(tokenizer.split_into_sentences(sentence))
    #tokenized_text = list(sentence.split_into_sentences(sentence))
    lemmas = []
    for sent in tokenized_text:
        for token in sent.split():
            if token in stop_words:
                continue
            lemma = naive_bin_lemma(token.lower())
            if lemma is not None:
                lemmas.append(lemma)
    return lemmas

In [143]:
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [145]:
load = True
if load:
    with open('data_words.pickle', 'rb') as fh:
        data_words = pickle.load(fh)

In [36]:
# Nú þegar keyrt
if not load:
    data_words = [tokenize_and_lemmatize(a) for a in tqdm.notebook.tqdm(data_train["text_tokenized"])]

In [37]:
if not load:
    with open('data_words.pickle', 'wb') as fh:
        pickle.dump(data_words, fh)

In [146]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])


['áheit', 'milljón', 'sólarhringur', 'undirskriftarsöfnun', 'hugmynd', 'flugfélag', 'maður', 'flug-míla_heildsöluverð', 'aðstandandi', 'söfnun', 'áform', 'stofnun', 'flugfélag', 'hugmynd', 'flugfélag', 'stefna', 'flugsamgöngur', 'áheit', 'vefsíða', 'áform', 'flugferð', 'fólk', 'kostur', 'farmiði', 'mínúta', 'brottför', 'fjármálastjóri', 'flugvélaframleiðandi', 'fólk', 'fótur', 'flugrekstur', 'flugrekstrarleyfi', 'fólk', 'þáttur', 'verkefni', 'flug-míla_heildsöluverð', 'átak', 'forsvarsmaður', 'verkefni', 'skarð', 'flugsamgöngur', 'heimasíða', 'félag', 'starfsmaður', 'kostur', 'áhugi', 'öryggi', 'farmiði', 'brottför', 'endurgjald', 'hugmynd', 'sólarhringur', 'skráning', 'milljón', 'forkaupsréttur', 'flug-míla', 'prósent', 'fyrirvari', 'flugfélag', 'ísland', 'félag', 'toppur', 'ís-lending']


In [147]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [148]:
data_words_bigrams = make_bigrams(tqdm.tqdm(data_words))

100%|███████████████████████████████████████████████████████████████████████| 544115/544115 [00:15<00:00, 35345.09it/s]


In [149]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [150]:
with open('data_words_bigrams.pickle', 'wb') as fh:
        pickle.dump(data_words_bigrams, fh)
with open('id2word.pickle', 'wb') as fh:
    pickle.dump(id2word, fh)
with open('corpus.pickle', 'wb') as fh:
        pickle.dump(corpus, fh)

In [151]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Ath eta, tengja náttúruverndarorð við ákveðinn hóp

In [152]:
lda_model.save("./Big_lda_model")

In [153]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.041*"maður" + 0.034*"kona" + 0.034*"fólk" + 0.029*"barn" + 0.021*"mynd" + '
  '0.017*"dagur" + 0.015*"fjölskylda" + 0.013*"saga" + 0.012*"rannsókn" + '
  '0.010*"viðtal"'),
 (1,
  '0.061*"lögregla" + 0.027*"dagur" + 0.023*"maður" + 0.023*"morgunn" + '
  '0.022*"klukka" + 0.018*"bíll" + 0.016*"höfuðborgarsvæði" + 0.015*"nótt" + '
  '0.014*"landslið" + 0.014*"land"'),
 (2,
  '0.032*"dagur" + 0.026*"leikmaður" + 0.023*"ísland" + 0.022*"kvöld" + '
  '0.021*"mínúta" + 0.016*"sigur" + 0.014*"mark" + 0.013*"deild" + '
  '0.013*"umferð" + 0.013*"tímabil"'),
 (3,
  '0.033*"dagur" + 0.021*"land" + 0.019*"fólk" + 0.018*"svæði" + 0.013*"íbúi" '
  '+ 0.013*"staður" + 0.010*"yfirvald" + 0.010*"ferðamaður" + 0.010*"fjöldi" + '
  '0.009*"sögn"'),
 (4,
  '0.034*"forseti" + 0.027*"bandaríki" + 0.027*"flokkur" + 0.025*"þingmaður" + '
  '0.020*"dagur" + 0.020*"forsætisráðherra" + 0.018*"ríkisstjórn" + '
  '0.017*"kosning" + 0.015*"þjóð" + 0.015*"íbúð"'),
 (5,
  '0.049*"mál" + 0.038*"frumvarp" + 

In [ ]:
if not load:
    #with open('data_words_bigrams', 'wb') as fh:
    #    pickle.dump(data_words_bigram, fh)
    with open('corpus', 'wb') as fh:
        pickle.dump(corpus, fh)

In [ ]:
[id2word[i[0]] for i in lda_model.get_topic_terms(1, topn=50)]

In [128]:
id2word.token2id["náttúruvernd"]

2642

In [145]:
# Af hverju sýnir þetta fyrir fæst orð?
lda_model.get_term_topics(id2word.token2id["neysla"])

[]

In [144]:
doc_lda[2][1]

[(16, [6, 1, 2, 9]),
 (46, [6]),
 (47, [6]),
 (48, [6]),
 (49, [6]),
 (50, [4]),
 (51, [6, 1]),
 (52, [6]),
 (53, [6]),
 (54, [6]),
 (55, [6]),
 (56, [1, 4]),
 (57, [6, 1, 7, 8]),
 (58, [6]),
 (59, [6]),
 (60, [4]),
 (61, [1, 4, 8]),
 (62, [6, 1]),
 (63, [1]),
 (64, [1, 4]),
 (65, [6]),
 (66, [1, 4, 8, 2]),
 (67, [1]),
 (68, [1, 4, 7])]

## Tilraunir

In [45]:
lda = gensim.models.ldamodel.LdaModel.load("small_lda_model")
#lda = gensim.models.ldamulticore.LdaModel.load("small_lda_model")

In [46]:
lda

In [47]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda.print_topics())
#doc_lda = lda[corpus]

  0%|                                                                                       | 0/544115 [08:36<?, ?it/s]

[(0,
  '0.043*"bíll" + 0.029*"myndband" + 0.024*"akureyri" + 0.020*"hátíð" + '
  '0.020*"samkomulag" + 0.018*"færsla" + 0.017*"ökumaður" + 0.017*"ummæli" + '
  '0.016*"nóvember" + 0.014*"hús"'),
 (1,
  '0.045*"ísland" + 0.015*"breyting" + 0.014*"verkefni" + 0.013*"fyrirtæki" + '
  '0.011*"áhrif" + 0.010*"sveitarfélag" + 0.009*"starfsmaður" + '
  '0.008*"frumvarp" + 0.008*"þjónusta" + 0.008*"einstaklingur"'),
 (2,
  '0.031*"svæði" + 0.022*"klukka" + 0.022*"íbúi" + 0.018*"samningur" + '
  '0.017*"nótt" + 0.017*"íbúð" + 0.016*"tilkynning" + 0.015*"höfuðborgarsvæði" '
  '+ 0.014*"verslun" + 0.014*"ferð"'),
 (3,
  '0.022*"hugmynd" + 0.021*"bók" + 0.016*"nemandi" + 0.013*"saga" + '
  '0.013*"umfjöllun" + 0.013*"háskóli" + 0.012*"skóli" + 0.012*"list" + '
  '0.011*"hlutverk" + 0.011*"sýning"'),
 (4,
  '0.025*"leikmaður" + 0.024*"mínúta" + 0.021*"kvöld" + 0.021*"ísland" + '
  '0.021*"dagur" + 0.018*"sigur" + 0.014*"mark" + 0.013*"umferð" + '
  '0.013*"deild" + 0.013*"tímabil"'),
 (5,
  '0.037*